## Homework 09:  Parallel Programming 02

## Due Date: Apr 19, 2023, 11:59pm

#### Firstname Lastname: 

#### E-mail: 

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process, say `Process 0`, independently draws $N$ samples uniformly on $[0, 1]$.

How do you explain the results?


**Instructions:** 
Your program should use MPI4PY and collective operations. 
Save your program as 2020_spring_sol09_pr01.py and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr01.py
```



---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


**Instructions:** 

Save your program as 2020_sol09_pr02.py; and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr02.py
```


**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?